# Assignment 05 Random search and hyperopt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 1. Load titanic.csv file.

In [2]:
data=pd.read_csv(r"C:\Users\USER\Downloads\titanic (2).csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Select ‘Pclass’,’SibSp’,’Parch’, ‘Fare’; columns as data. Select ‘Survived’ column as label.

In [4]:
X=data[['Pclass','SibSp','Parch','Fare']]
y=data['Survived']

## 3. Apply SVM to classify the data.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
model=SVC()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Accuracy of SVM model : ",round(100*accuracy_score(y_test,y_pred),2))

Accuracy of SVM model :  65.36


## 4. Use 5-Fold Cross Validation to generalise the model accuracy.

In [8]:
from sklearn.model_selection import cross_val_score,KFold
model = SVC()
kf=KFold(n_splits=5)
score=cross_val_score(model,X,y,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.62011173 0.66853933 0.66853933 0.66292135 0.70786517]
Average Cross Validation score :0.6655953800765803


## 5. Use random search and hyperopt packages to identify the best possible SVM model from the following search space.

'C': [0.1, 1, 10, 100, 1000]

'gamma': [1, 0.1, 0.01, 0.001, 0.0001]

'kernel': ['rbf']

### Random Search

In [9]:
parameters={'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf']}

In [10]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  69.94385895794348
Best Params:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 100}


### Hyperopt

In [11]:
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials

space = {
      'C': hp.choice('C',[0.1, 1, 10, 100, 1000]),
      'kernel': hp.choice('kernel',['rbf']),
      'gamma': hp.choice('gamma',[1, 0.1, 0.01, 0.001, 0.0001])
      }

In [12]:
# define objective function

def hyperparameter_tuning(params):
    clf = SVC()
    acc = cross_val_score(clf, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [13]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [00:33<00:00,  1.51trial/s, best loss: -0.6678865105768628]
Best: {'C': 4, 'gamma': 0, 'kernel': 0}


In [14]:
trials.results

[{'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status': 'ok'},
 {'loss': -0.6678865105768628, 'status':

In [15]:
trials.losses()

[-0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.6678865105768628,
 -0.667886